In [2]:
import pandas as pd
import numpy as np

In [3]:
movies = pd.read_csv("/Users/annavlasova/Downloads/content_groups.csv", sep=';')
print(movies.head())

                               uuid  \
0  4a1efb3fbddb53598d00e088665f09b4   
1  432ed9f64c988118911c266d66d4437a   
2  46d1499bbd44712ba0f1e79e01cb5be5   
3  4facb695e3144238aa6ca07e83eb2d44   
4  4b564c5dc2b71baeb9057ba10ca270cf   

                                            actors director  \
0  О’Ши Джексон мл., Кори Хоукинс, Джейсон Митчелл     None   
1    Алекс Петтифер, Габриэлла Уайлд, Брюс Гринвуд     None   
2          Жан Дюжарден, Беренис Бежо, Джон Гудман     None   
3    Дж.К. Симмонс, Чарли Бьюли, Салливан Степлтон     None   
4        Джеймс Пьюрфой, Пол Джаматти, Брайан Кокс     None   

                                           producers  \
0            Мэтт Альварес, Скотт Бернштейн, Др. Дре   
1         Памела Эбди, Дж. Майлз Дэйл, Трэйси Фалько   
2     Жереми Бюрдек, Антуан де Казотте, Дэниэл Делюм   
3       Рафаэль Бенолье, Люк Бессон, Фабрис Дельвиль   
4  Гленн Кендрик Экерманн, Кристиан Арнольд-Бютел...   

                                              gen

# Векторизация жанров, вычисление похожих фильмов с помощью cos similarity

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0)

movies['genres'] = movies['genres'].apply(lambda x: "None" if x is np.nan else x)
tfidf_matrix = tf.fit_transform(movies['genres'])


In [5]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [6]:
titles = movies['name']
indices = pd.Series(movies.index, index=movies['name'])

def genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [7]:
genre_recommendations("Мадагаскар").head(20)


19                                          Нико 2
48                                  Реальная белка
190                                     Мадагаскар
318                         Тайная жизнь насекомых
488                  Снупи и мелочь пузатая в кино
492                                 Папа-мама гусь
727                          Большой собачий побег
780                        Индюки: Назад в будущее
800     Ледниковый период 4: Континентальный дрейф
924                                   Мадагаскар 3
1152                             Смешарики. Начало
1192                                 Подводная эра
1382                                  Мадагаскар 2
1485                             Ворона-проказница
1558                      Пчёлка Майя и Кубок мёда
1607           Ледниковый период 3: Эра динозавров
1617                             Смешарики. Дежавю
512                                Гладиаторы Рима
874                                       От винта
1194                           

___

# kNN (не учитывает actors, producers and director пока что)

In [8]:
movies.columns
for col in movies.columns:
    movies[col] = movies[col].apply(lambda x: 0 if (x == "None" or x is np.nan) else x)

In [9]:
movies_features = pd.concat([#movies["actors"].str.get_dummies(sep=","),
                           #movies["director"],
                            #movies["producers"].str.get_dummies(sep=","),
                            movies["genres"].str.get_dummies(sep=","),
                            movies["countries"].str.get_dummies(sep=","),
                            movies[["kinopoisk_rating"]], movies[["imdb_rating"]],
                            movies[["year"]], movies["duration"]],axis=1)

# anime["name"] = anime["name"].map(lambda name:re.sub('[^A-Za-z0-9]+', " ", name))
movies_features.head()

,аниме,биография,боевик,вестерн,военный,детектив,детский,документальный,драма,история,...,Чили,Швейцария,Швеция,ЮАР,Южная Корея,Япония,kinopoisk_rating,imdb_rating,year,duration
0,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,7.847,7.9,2015,8840
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,6.707,6.3,2014,6275
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,7.941,7.9,2011,5766
3,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,5.418,5.6,2017,6312
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,6.666,6.2,2010,6948


In [10]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
movies_features = min_max_scaler.fit_transform(movies_features)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [11]:
movies_features
movies_features = np.round(movies_features,2)

print(movies_features)

[[0.   1.   0.   ... 0.86 0.96 0.73]
 [0.   0.   0.   ... 0.68 0.94 0.51]
 [0.   0.   0.   ... 0.86 0.9  0.46]
 ...
 [0.   0.   0.   ... 0.82 0.78 0.8 ]
 [0.   0.   0.   ... 0.65 0.93 0.39]
 [0.   0.   0.   ... 0.54 0.99 0.41]]


In [12]:
from sklearn.neighbors import NearestNeighbors

nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(movies_features)

In [13]:
distances, indices = nbrs.kneighbors(movies_features)

In [14]:
def get_index_from_name(name):
    return movies[movies["name"]==name].index.tolist()[0]

In [15]:
def print_similar_movies(query=None,id=None):
    if id:
        for id in indices[id][1:]:
            print(movies.ix[id]["name"])
    if query:
        found_id = get_index_from_name(query)
        for id in indices[found_id][1:]:
            print(movies.ix[id]["name"])

In [16]:
print_similar_movies(query="Моя ужасная няня")

Ночь в музее
Пингвины мистера Поппера
Эван Всемогущий
Доктор Дулиттл
Оторва
Гринч – похититель Рождества
Третий лишний
Лжец, лжец
Дом с паранормальными явлениями


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


___

# Collaborative filtering с помощью SVD разложения 

In [24]:
ratings_data

,user_id,uuid_film,monetization_model: EST/TVOD/SVOD/AVOD,price,service_id: 10/25,expiration_date (TVOD)
0,1762,46c25e923a9c4caaa67990a5f08512fa,EST,299,10,None
1,1762,4bec85e370c5f10b8a828c53b1e6349b,EST,299,10,None
2,1762,47cdb4a90fc23fe1a4a7863b4d9f62ac,EST,299,10,None
3,213,47cdb4a90fc23fe1a4a7863b4d9f62ac,EST,299,10,None
4,213,4c15fa9fe9d5df8ebce51232f143eb8f,EST,299,10,None
5,1762,4c15fa9fe9d5df8ebce51232f143eb8f,EST,299,10,None
6,213,47c6cc97be699bccbdf2b5d784949b31,EST,299,10,None
7,213,4a58b40bd087c5b2b4a8d25adf6bbcab,EST,299,10,None
8,207,4c15fa9fe9d5df8ebce51232f143eb8f,EST,299,10,None
9,207,46c25e923a9c4caaa67990a5f08512fa,EST,299,10,None


In [25]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [26]:
le=le.fit(ratings_data["uuid_film"])

In [27]:
le.classes_

array(['400258b134c6fa3f915e74a48d29721d',
       '4003120ebc58eec898fe1af3104537b8',
       '4006a98397df5fde8093993960326c9c', ...,
       '4ff9b440cf239480ac96ee6ee2cae8c5',
       '4ffb21946fb420fe8539ff99a83f543b',
       '4fff65d9332c18fa8505d3e3b5e504d5'], dtype=object)

In [28]:
movies["uuid"]=le.transform(movies["uuid"])

In [29]:
ratings_data["uuid_film"] = le.transform(ratings_data["uuid_film"])

In [30]:
le_users = preprocessing.LabelEncoder()
le_users=le_users.fit(ratings_data["user_id"])
le_users.classes_

array([    50,     56,    104, ..., 917009, 917024, 917029])

In [31]:
ratings_data["user_id"] = le_users.transform(ratings_data["user_id"])

In [32]:
ratings_data["rating"] = 1
ratings_data

,user_id,uuid_film,monetization_model: EST/TVOD/SVOD/AVOD,price,service_id: 10/25,expiration_date (TVOD),rating
0,31,693,EST,299,10,None,1
1,31,1273,EST,299,10,None,1
2,31,817,EST,299,10,None,1
3,6,817,EST,299,10,None,1
4,6,1288,EST,299,10,None,1
5,31,1288,EST,299,10,None,1
6,6,814,EST,299,10,None,1
7,6,1095,EST,299,10,None,1
8,5,1288,EST,299,10,None,1
9,5,693,EST,299,10,None,1


In [33]:
n_users = ratings_data.user_id.unique().shape[0]
n_movies = ratings_data.uuid_film.unique().shape[0]

In [34]:
print(n_users, n_movies)

31541 1713


In [35]:
ratings_data = ratings_data.drop(columns=["monetization_model: EST/TVOD/SVOD/AVOD", 
                                         "price", "service_id: 10/25", "expiration_date (TVOD)"])

In [36]:
ratings_data.drop(columns=['rating'])

,user_id,uuid_film
0,31,693
1,31,1273
2,31,817
3,6,817
4,6,1288
5,31,1288
6,6,814
7,6,1095
8,5,1288
9,5,693


In [37]:
ratings_data.drop_duplicates()

,user_id,uuid_film,rating
0,31,693,1
1,31,1273,1
2,31,817,1
3,6,817,1
4,6,1288,1
5,31,1288,1
6,6,814,1
7,6,1095,1
8,5,1288,1
9,5,693,1


In [38]:
ratings_data['rating']=1
ratings_data.head()

,user_id,uuid_film,rating
0,31,693,1
1,31,1273,1
2,31,817,1
3,6,817,1
4,6,1288,1


In [39]:
Ratings = ratings_data.pivot_table(index = 'user_id', columns ='uuid_film', values = 'rating').fillna(0)
Ratings.head()

uuid_film,0,1,2,3,4,5,6,7,8,9,...,1703,1704,1705,1706,1707,1708,1709,1710,1711,1712
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [40]:
Ratings.info

<bound method DataFrame.info of uuid_film  0     1     2     3     4     5     6     7     8     9     ...   \
user_id                                                                ...    
0           0.0   0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...    
1           0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...    
2           0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...    
3           0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...    
4           0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...    
5           0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...    
6           0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...    
7           0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...    
8           0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...    
9           0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...    
10          0.0   0.

In [43]:
sparsity = 1.0 - len(ratings_data) / float(n_users * n_movies)
print('The sparsity level of dataset is ', str(sparsity * 100), '%')

The sparsity level of dataset is  99.90117293379924 %


In [42]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(Ratings, k = 100)

In [44]:
sigma = np.diag(sigma)


In [45]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)


In [46]:
preds = pd.DataFrame(all_user_predicted_ratings, columns = Ratings.columns)
preds.head()

uuid_film,0,1,2,3,4,5,6,7,8,9,...,1703,1704,1705,1706,1707,1708,1709,1710,1711,1712
0,0.028797,0.022811,1.078163,0.005310,0.031998,-0.007533,0.010773,0.037128,-8.521957e-18,0.000703,...,0.000823,0.044627,-0.000330,2.062926e-17,-3.986032e-18,-0.002916,0.011546,0.027185,-0.001430,0.017747
1,-0.000931,0.005939,-0.000867,0.001834,0.001647,0.001312,0.000483,-0.000100,-1.401961e-18,-0.000137,...,-0.000562,0.002577,0.000826,2.404627e-18,-7.859447e-19,-0.000395,-0.000348,-0.000887,1.000197,0.000609
2,0.000249,0.005131,-0.002092,0.008228,-0.000062,-0.001565,-0.003081,-0.008251,-6.664610e-19,-0.000381,...,0.001227,0.023490,-0.000086,1.723616e-18,-9.080200e-19,-0.000676,0.000585,0.000181,0.001742,-0.007368
3,0.022100,-0.040053,0.004473,0.082719,0.069868,0.009096,-0.016804,0.046526,1.036786e-17,-0.003109,...,-0.003557,0.177444,0.000217,-1.208468e-17,6.485311e-18,0.003764,-0.007015,0.019987,1.007331,1.149556
4,-0.002544,-0.000720,0.012141,0.069212,0.026547,0.025142,0.031026,0.035046,-1.235131e-17,-0.002821,...,0.000057,-0.080932,0.000407,2.409651e-17,-6.278364e-18,-0.000330,0.000198,-0.002849,1.012165,1.050695


In [47]:
preds.info

<bound method DataFrame.info of uuid_film      0         1         2         3         4         5     \
0          0.028797  0.022811  1.078163  0.005310  0.031998 -0.007533   
1         -0.000931  0.005939 -0.000867  0.001834  0.001647  0.001312   
2          0.000249  0.005131 -0.002092  0.008228 -0.000062 -0.001565   
3          0.022100 -0.040053  0.004473  0.082719  0.069868  0.009096   
4         -0.002544 -0.000720  0.012141  0.069212  0.026547  0.025142   
5          0.008259  0.015804 -0.007713  0.056376  0.019702  0.013754   
6          0.004219 -0.006282  0.014529  0.010138  0.002785  0.011400   
7         -0.000131  0.000083 -0.001064  0.000353  0.000308  0.000193   
8         -0.000086 -0.000061 -0.000578  0.000364  0.000474 -0.000224   
9         -0.000065 -0.005149  0.001436  0.000736  0.001956 -0.001613   
10        -0.001647  0.009337 -0.002655 -0.001834  0.005702  0.001915   
11        -0.000981 -0.011094  0.000768  0.003714  0.000905 -0.002073   
12         0.001532

In [49]:
def recommend_movies(predictions, userID, num_recommendations):
    # get users's rated films 
    user_row_number = le_users.transform([userID])[0] # tranform of label encoder
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) 

    sorted_user_predictions=pd.merge(sorted_user_predictions.to_frame(), movies, left_on="uuid_film", right_on="uuid")
    
#     print(sorted_user_predictions)
#     movies_predictions = pd.DataFrame()
#     movies_predictions['uuid'] = le.inverse_transform(sorted_user_predictions.index)
#     movies_predictions['accuracy'] = sorted_user_predictions.tolist()
#     print(movies_predictions)
    
    user_data = ratings_data[ratings_data.user_id == user_row_number]
#     print(user_data)
    user_full = (user_data.merge(movies, how = 'left', left_on = 'uuid_film', right_on = 'uuid').
                     sort_values(['rating'], ascending=False)
                 )
    print(user_full)

    print ('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    recommendations = (movies[~movies['uuid'].isin(user_full['uuid_film'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'uuid',
               right_on = 'uuid').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )
#     print('\n', recommendations)
    return user_full, recommendations

In [50]:
user_rated, recommend = recommend_movies(preds, 2543, 2)

    user_id  uuid_film  rating    uuid  \
0        34       1139       1  1139.0   
46       34        875       1   875.0   
33       34       1188       1  1188.0   
34       34        930       1   930.0   
35       34         36       1    36.0   
36       34        150       1   150.0   
37       34         37       1    37.0   
38       34        397       1   397.0   
39       34        131       1   131.0   
40       34        673       1   673.0   
41       34        410       1   410.0   
42       34        684       1   684.0   
43       34       1595       1  1595.0   
44       34        219       1   219.0   
45       34        304       1   304.0   
47       34       1458       1  1458.0   
1        34       1659       1  1659.0   
48       34        905       1   905.0   
49       34       1159       1  1159.0   
50       34        645       1   645.0   
51       34        489       1   489.0   
52       34       1007       1  1007.0   
53       34        398       1   3

In [51]:
user_rated

,user_id,uuid_film,rating,uuid,actors,director,producers,genres,countries,name,kinopoisk_rating,imdb_rating,year,duration
0,34,1139,1,1139.0,"Майкл Кейн, Мелани Лоран, Морган Фриман",0.0,"Алекс Куртцман, Бобби Коэн, Роберто Орси","триллер, криминал, детектив","США, Франция",Иллюзия обмана,7.717,7.3,2013.0,6637.0
46,34,875,1,875.0,"Кира Найтли, Джеймс МакЭвой, Сирша Ронан",0.0,"Тим Беван, Лиза Чейсин, Ричард Эйр","драма, мелодрама, детектив, военный","Великобритания, Франция, США",Искупление,7.98,7.8,2007.0,7067.0
33,34,1188,1,1188.0,"Люк Эванс, Сара Гадон, Доминик Купер",0.0,"Джозеф М. Карачиоло мл., Майкл Де Лука, Джон Д...","ужасы, фэнтези, боевик, драма, военный","США, Япония",Дракула,6.559,6.3,2014.0,5515.0
34,34,930,1,930.0,"Клайв Оуэн, Клэр-Хоуп Эшити, Джулианна Мур",0.0,"Марк Абрахам, Эрмиан Бернштейн, Томас А. Блисс","фантастика, триллер, драма","США, Великобритания, Япония",Дитя человеческое,7.369,7.9,2006.0,6543.0
35,34,36,1,36.0,"Эдди Редмэйн, Алисия Викандер, Маттиас Шонартс",0.0,"Тим Беван, Лиза Чейсин, Эрик Феллнер","драма, мелодрама, биография, история","Великобритания, США, Германия, Дания, Бельгия",Девушка из Дании,6.887,7.1,2015.0,7164.0
36,34,150,1,150.0,"Джим Керри, Тейлор Момсен, Келли",0.0,"Линда Филдс, Брайан Грейзер, Тодд Халлоуэлл","фэнтези, комедия, семейный","США, Германия",Гринч – похититель Рождества,6.81,6.1,2000.0,6293.0
37,34,37,1,37.0,"Джон Кьюсак, Кэмерон Диаз, Нед Беллами",0.0,"Стив Голин, Чарли Кауфман, Майкл Кун","фэнтези, драма",США,Быть Джоном Малковичем,7.243,7.8,1999.0,6763.0
38,34,397,1,397.0,"Эрика Линдбек, Робби Дэймонд, Кимберли Вудс",0.0,"Марианн Кулберт, Челла Николс Даффи, Каллен Кэген","мультфильм, семейный",США,Барби и космическое приключение,5.727,5.9,2016.0,4765.0
39,34,131,1,131.0,"Джастин Лонг, Дженна Дуан-Татум, Ченнинг Татум",0.0,"Вик Годфри, Марти Бауэн, Рид Каролин","драма, мелодрама, комедия",США,10 лет спустя,5.991,6.1,2011.0,5790.0
40,34,673,1,673.0,"Карл Урбан, Розамунд Пайк, Деобиа Опарей",0.0,"Лоренцо Ди Бонавентура, Лаура Холстейн, Дэвид ...","ужасы, фантастика, боевик, приключения","Великобритания, Чехия, Германия, США",Doom,5.86,5.2,2005.0,6266.0


In [52]:
recommend

,uuid,actors_x,director_x,producers_x,genres_x,countries_x,name_x,kinopoisk_rating_x,imdb_rating_x,year_x,...,Predictions,actors_y,director_y,producers_y,genres_y,countries_y,name_y,kinopoisk_rating_y,imdb_rating_y,year_y
301,30,"Вин Дизель, Пол Уокер, Джейсон Стэйтем",0,"Брэндон Биртелл, Вин Дизель, Майкл Фоттрел","боевик, триллер, криминал",США,Форсаж 7,7.035,7.2,2015,...,0.844681,"Вин Дизель, Пол Уокер, Джейсон Стэйтем",0,"Брэндон Биртелл, Вин Дизель, Майкл Фоттрел","боевик, триллер, криминал",США,Форсаж 7,7.035,7.2,2015
1618,1165,"Майкл Дж. Фокс, Кристофер Ллойд, Лиа Томпсон",0,"Нил Кэнтон, Боб Гейл, Кэтлин Кеннеди","фантастика, боевик, комедия, приключения, семе...",США,Назад в будущее 2,8.287,7.8,1989,...,0.490348,"Майкл Дж. Фокс, Кристофер Ллойд, Лиа Томпсон",0,"Нил Кэнтон, Боб Гейл, Кэтлин Кеннеди","фантастика, боевик, комедия, приключения, семе...",США,Назад в будущее 2,8.287,7.8,1989


___